In [6]:
import pandas as pd
import numpy as np

In [12]:
def process_all_tickets(all_tickets_path, chair_mapping_path, sf_path):
    # Load all_tickets CSV
    all_tickets = pd.read_csv(all_tickets_path)

    # Rename columns
    all_tickets = all_tickets.rename(columns={
        'Product Id': 'Product ID',
        'Product': 'Asset Name',
        'Id': 'Roboticket ID',
        'Price area': 'Price Area Name',
        'Ticket price types': 'Price Name',
        'Base price': 'Base Price',
        'Area': 'Sector',
        'Type': 'Operation Name',
        'Date.1': 'Product Date',
        'User Id': 'Owner User ID',
        'assign using  ID number': 'Owner Identifier'
    })

    # Add missing columns
    new_columns = [
        'Season ID', 'From', 'To', 'Stadium ID', 'Product Type', 'Product Name', 'Operation', 'Return Type', 'Return ID',
        'Platform', 'Entrance Text', 'Seat ID', 'Price Modifier Value', 'Access Token', 'Ticket ID', 'Transaction Owner',
        'Description', 'Account', 'Account ID', 'Contact', 'Product', 'Purchaser Account',
        'Return Asset', 'Status', 'Is Ticket', 'Is Season Ticket', 'Created By', 'Last Modified By'
    ]
    for col in new_columns:
        if col not in all_tickets.columns:
            all_tickets[col] = None

    # Reorder columns
    desired_columns = [
        'Product ID' ,'Season ID', 'Asset Name', 'Roboticket ID', 'Price', 'Price Name', 'Price Area Name', 'Base Price', 'From', 'To',
        'Stadium ID', 'Sector', 'Row', 'Number', 'Product Type', 'Operation Name', 'Product Name', 'Product Date',
        'Operation', 'Return Type', 'Return ID', 'Owner User ID', 'Owner Identifier', 'Platform', 'Entrance Text',
        'Seat ID', 'Price Modifier Value', 'Access Token', 'Ticket ID', 'Transaction Owner', 'Purchaser Account', 'Stadium',
        'Description', 'Account', 'Account ID', 'Contact', 'Product', 'Transaction', 'Return Asset', 'Status', 'Is Ticket',
        'Is Season Ticket', 'Created By', 'Last Modified By'
    ]
    for col in desired_columns:
        if col not in all_tickets.columns:
            all_tickets[col] = None
    all_tickets = all_tickets[desired_columns]

    # Set default values
    all_tickets['Season ID'] = 24
    all_tickets['Product Name'] = all_tickets['Asset Name']
    all_tickets['Product Type'] = 'Match'
    all_tickets['Operation Name'] = 'Sale'
    all_tickets['Purchaser Account'] = all_tickets['Owner User ID']
    all_tickets['Is Ticket'] = True
    all_tickets['Is Season Ticket'] = False
    all_tickets['Stadium ID'] = np.where(
        all_tickets['Stadium'] == 'פיס ארנה', 1, all_tickets['Stadium ID']
    )

    # Map Operation Name to Operation Code
    conditions = [
        all_tickets['Operation Name'] == 'Sale',
        all_tickets['Operation Name'] == 'Return'
    ]
    choices = [1, 2]
    all_tickets['Operation'] = np.select(conditions, choices, default=all_tickets['Operation'])

    # Define mapping function for Hebrew sectors
    def map_sector_to_section_name(sector):
        sector = str(sector).strip()
        if sector.startswith('אולם'):
            number = ''.join(filter(str.isdigit, sector))
            return f'Court_{number}' if number else None
        elif 'תא צפוני' in sector:
            return 'Suite_21'
        elif 'תא צפיה' in sector:
            number = ''.join(filter(str.isdigit, sector))
            if number and 1 <= int(number) <= 16:
                return f'Suite_{number}'
        elif 'פרקט דרומי' in sector:
            return 'Court_side_17'
        elif 'פרקט צפוני' in sector:
            return 'Court_side_19'
        elif 'פרקט מרכזי' in sector or 'פרקט מרכז' in sector:
            return 'Court_side_18'
        elif 'גלריה' in sector:
            number = ''.join(filter(str.isdigit, sector))
            return f'Gallery_{number}' if number else None
        elif 'Courtyard' in sector:
            return 'Unmarked_12'
        return None

    # Map section name
    all_tickets['section_name'] = all_tickets['Sector'].apply(map_sector_to_section_name)
    all_tickets['Row'] = all_tickets['Row'].astype(str).str.strip()
    all_tickets['Number'] = all_tickets['Number'].astype(str).str.strip()

    # Load chair mapping
    chair_mapping = pd.read_excel(chair_mapping_path)
    chair_mapping['row_number'] = chair_mapping['row_number'].astype(str).str.strip()
    chair_mapping['seat_number'] = chair_mapping['seat_number'].astype(str).str.strip()

    # Merge to get Seat ID
    all_tickets = all_tickets.merge(
        chair_mapping[['seat_id', 'section_name', 'row_number', 'seat_number']],
        how='left',
        left_on=['section_name', 'Row', 'Number'],
        right_on=['section_name', 'row_number', 'seat_number']
    )

    all_tickets['Seat ID'] = all_tickets['seat_id']
    all_tickets.drop(columns=['seat_id', 'section_name', 'row_number', 'seat_number'], inplace=True)

    # Load SF data
    sf = pd.read_csv(sf_path)
    all_tickets_ids = all_tickets[['Owner Identifier', 'Account ID']].copy()
    merged_ids = all_tickets_ids.merge(
        sf[['id__c', 'AccountId']].astype(str).apply(lambda x: x.str.strip()),
        how='left',
        left_on='Owner Identifier',
        right_on='id__c'
    )
    merged_ids['Account ID'] = merged_ids['Account ID'].fillna(merged_ids['AccountId'])
    merged_ids.drop(columns=['id__c', 'AccountId'], inplace=True)
    all_tickets['Account ID'] = merged_ids['Account ID']

    return all_tickets


In [13]:
df = process_all_tickets(
    all_tickets_path='all_tickets.csv',
    chair_mapping_path='chair_mapping_20240801.xlsx',
    sf_path='extract.csv'
)

/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_5654/150659598.py:3: DtypeWarning: Columns (0,12,32,48) have mixed types. Specify dtype option on import or set low_memory=False.
  all_tickets = pd.read_csv(all_tickets_path)


In [14]:
df.iloc[5000]

Product ID                                    1694
Season ID                                       24
Asset Name              Round 2: Hapoel Tel Aviv 🚗
Roboticket ID                            3064027.0
Price                                         40.0
Price Name                                   Child
Price Area Name                                  D
Base Price                                    40.0
From                                          None
To                                            None
Stadium ID                                    None
Sector                                      אולם 6
Row                                             21
Number                                          41
Product Type                                 Match
Operation Name                                Sale
Product Name            Round 2: Hapoel Tel Aviv 🚗
Product Date                            2024-10-13
Operation                                        1
Return Type                    

In [15]:
df.to_excel('tickets2425.xlsx')